In [1]:
import os
import os.path as ospath
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil
import glob
from tqdm.notebook import tqdm

from picasso.picasso import io
from picasso.picasso.postprocess import link, compute_dark_times
from picasso.picasso.gui.render import estimate_kinetic_rate, fit_cum_exp



In [2]:
def load_ring_data_df(dirname, filename):
    file = ospath.join(dirname, filename)

    try: 
        df = pd.read_pickle(file)
    except FileNotFoundError:
        print("The file {} does not exist.".format(filename))
        return None
    else: 
        print("The file {} was loaded.".format(filename))
        return df
    
def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")
        

    if hasattr(locs, "x_pick_rot"):

        # convert px to nm
        locs.x *= pixelsize
        locs.y *= pixelsize
        locs.x_pick_rot *= pixelsize
        locs.y_pick_rot *= pixelsize
        
        return locs, info, pixelsize
    else:
        print('x_pick_rot column is missing!')
        return None, None, None

        
    #return locs, info, pixelsize


def plot_cum_exp(pooled_locs, fit_result_len, fit_result_dark, locs_description, ax=None):
    
    color = blue

    if ax is None:
        ax = plt.gca()
    
    data = pooled_locs.dark
    data.sort()
    y = np.arange(1, len(data) + 1)
       
    a = fit_result_dark.best_values["a"]
    t = fit_result_dark.best_values["t"]
    c = fit_result_dark.best_values["c"]

    ax.set_title(
        "Dark time (cumulative) - {} \n"
        r"$Fit: {:.2f}\cdot(1-exp(x/{:.2f}))+{:.2f}$".format(locs_description, a, t, c),loc="left",fontsize=14)
    data = pooled_locs.dark
    data.sort()
    y = np.arange(1, len(data) + 1)

    ax.semilogx(data, y, c=color, label="Data")
    ax.semilogx(data, fit_result_dark.best_fit, c=red, label="Fit")
    ax.legend(loc="best")
    ax.set_xlabel("Duration (frames)")
    ax.set_ylabel("Frequency")
    #ax.set_xlim(1,data.max()+1)
    
    return ax


In [3]:
path = r'X:\users\kcramer\sporPAINT\ZapA\ZapA_picked_rings'

# consider locs within +- dR around the fitted ring radius
dR = 100

# linking of locs
max_dist = 130 #nanometer
max_dark_time = 15 #frames



# load ring fit results
df_ring_data = load_ring_data_df(path, "ring_data_filter.pkl")
filtered = True
if df_ring_data is None:
    df_ring_data = load_ring_data_df(path, "ring_data.pkl")
    filtered = False
print(df_ring_data.keys())


The file ring_data_filter.pkl was loaded.
Index(['fov_id', 'cell_type', 'filename', 'group', 'residual', 'radius',
       'angle_between_ring_and_coverslip', 'Sigma', 'n_events', 'mean_bright',
       'mean_dark', 'fit_bright', 'fit_dark', 'filter_passed'],
      dtype='object')


In [4]:
orientation = 'yz' # use yz, but also xy files contain info for filtering

analysis_folder = os.path.join(path, "analysis")
excluded_folder = os.path.join(path, 'analysis', 'excluded_filter')

# image export settings
img_format = ".png"
dpi = 100

# define colors:
blue = "#4C72B0"
orange = "#DD8452"
red = "#C44E52"
gray = "#90A8CE"

# prepare angle coordinates for plotting of rings check:
# angles from -180 to 180
bin_width = 360/12
bins = np.arange(-180, 180+bin_width, bin_width)

ring_data = []
# add filter_passed column:
df_ring_data['filename_ring'] =  'fov_' + df_ring_data['fov_id'].astype(str) +'_' + df_ring_data['cell_type'].astype(str) +'_pick_' + df_ring_data['group'].astype(str) +'_ring_' + '0' +'_rot_' + orientation + '_update.hdf5'
for index, ring in tqdm(df_ring_data.iterrows(), desc="Processing rings", total = df_ring_data.shape[0]):
    plt.close('all')
    locs, info, pixelsize = load_data(os.path.join(path, 'analysis', 'ring_locs', ring['filename_ring'] ))
    
    radius_fit = ring['radius']
    radius_min = radius_fit - dR
    radius_max = radius_fit + dR
    
    
    N_locs = len(locs)

    ################
    # Get locs on ring
    ################
    locs_ring = locs[(locs['radius']>radius_min) & (locs['radius']<radius_max)]
    df_locs_ring = pd.DataFrame.from_records(locs_ring)
    locs_not_ring = locs[(locs['radius']<=radius_min) | (locs['radius']>=radius_max)]
    df_locs_not_ring = pd.DataFrame.from_records(locs_not_ring)
    


    ################
    # Link locs & calculate dark and bright times for all locs
    ################
    # link the isolated locs
    r_locs_linked_all = link(locs, info, max_dist, max_dark_time)

    # extract number of events
    ring_n_events_all = len(r_locs_linked_all)
    
    # compute kinetics
    kinetics_all = compute_dark_times(r_locs_linked_all)
    #print('kinetics', kinetics)

    # compute mean bright and dark times
    ring_kinetics_all = {"bright":np.nanmean(kinetics_all.len),
                         "dark":np.nanmean(kinetics_all.dark)}

    fit_result_len_all = fit_cum_exp(kinetics_all.len)
    fit_result_dark_all = fit_cum_exp(kinetics_all.dark)
    
    ################
    # Link locs & calculate dark and bright times for locs on ring
    ################
    # link the isolated locs
    r_locs_linked_ring = link(locs_ring, info, max_dist, max_dark_time)

    # extract number of events
    ring_n_events_ring = len(r_locs_linked_ring)
    
    # compute kinetics
    kinetics_ring = compute_dark_times(r_locs_linked_ring)
    #print('kinetics', kinetics)

    # compute mean bright and dark times
    ring_kinetics_ring = {"bright":np.nanmean(kinetics_ring.len),
                         "dark":np.nanmean(kinetics_ring.dark)}

    fit_result_len_ring = fit_cum_exp(kinetics_ring.len)
    fit_result_dark_ring = fit_cum_exp(kinetics_ring.dark)
    
    
    ################
    # (5.2) append data into large array
    ################

    ring_kinetics_fit_all = {"bright":np.nanmean(fit_result_len_all.best_values["t"]),
                             "dark":np.nanmean(fit_result_dark_all.best_values["t"])}
    ring_kinetics_fit_ring = {"bright":np.nanmean(fit_result_len_ring.best_values["t"]),
                             "dark":np.nanmean(fit_result_dark_ring.best_values["t"])}
    
    new_ring_data = [ring['fov_id'], #running file index
                      ring['cell_type'], # veg or spore
                      ring['filename'], #filename
                      ring['group'], #pick number
                      ring['residual'], #residual of ring fit
                      ring['radius'], #fitted radius of spore (nm)
                      ring['angle_between_ring_and_coverslip'], #anle between spore ring and coverslip (°)
                      ring['Sigma'], #FWHM
                      ring_n_events_all, #number of locs in spore ring
                      ring_kinetics_all["bright"], #mean bright time of spore ring (frames)
                      ring_kinetics_all["dark"], #mean dark time of spore ring (frames)
                      ring_kinetics_fit_all["bright"], #cumulative mean bright times of spore (frames)
                      ring_kinetics_fit_all["dark"], #cumulative mean dark times of spore (frames)
                      ring_n_events_ring, #number of locs in spore ring
                      ring_kinetics_ring["bright"], #mean bright time of spore ring (frames)
                      ring_kinetics_ring["dark"], #mean dark time of spore ring (frames)
                      ring_kinetics_fit_ring["bright"], #cumulative mean bright times of spore (frames)
                      ring_kinetics_fit_ring["dark"] #cumulative mean dark times of spore (frames)
                     ]
    if filtered:
        new_ring_data.append(ring['filter_passed'])
        
    ring_data.append(new_ring_data)
    
    
    ################
    # Plot
    ################
    
    
    # set up plot illustrating ring +- dR
    fig = plt.figure(figsize=(20, 10), constrained_layout=True)
    gs = fig.add_gridspec(1,3)
    fig.suptitle(("FOV {}, {}, Pick {} - $1/\tau _d$ and $1/\tau _b$ for locs on ring \n"
                  "File: {}\n").format(ring['fov_id'], ring['cell_type'] , ring['group'], ring['filename']), 
                 fontsize=16, 
                 ha="center")

    # set maximum radius for plot such that 99% of locs are shown
    cutoff = 1

    radius_sort = np.sort(locs['radius'])
    radius_limit = radius_sort[int(cutoff*len(radius_sort))-1]

    ## ring criterium
    ax_1 = fig.add_subplot(gs[0, 0], projection='polar')
    
    ax_1.set_xticks(bins[1:]/180*np.pi)
    ax_1.set_thetalim(-np.pi,np.pi)

    #ring of fit radius
    coords_angle_fit = np.arange(-np.pi,np.pi,0.01)
    coords_radius_fit = np.full(len(coords_angle_fit), radius_fit)
    coords_radius_min = np.full(len(coords_angle_fit), radius_min)
    coords_radius_max = np.full(len(coords_angle_fit), radius_max)
    
    ax_1.fill_between(coords_angle_fit,radius_min, radius_max, color='red', alpha=0.2)

    ax_1.plot(coords_angle_fit, coords_radius_fit, c='red', lw=2)
    ax_1.plot(coords_angle_fit, coords_radius_min, c='red', lw=1)
    ax_1.plot(coords_angle_fit, coords_radius_max, c='red', lw=1)
    
    # locs
    #ax_1.scatter(locs['angle']/180*np.pi, locs['radius'], alpha=0.5)
        # locs
    ax_1.scatter(df_locs_not_ring['angle']/180*np.pi, df_locs_not_ring['radius'], alpha=0.1, c='#1f77b4')
    ax_1.scatter(df_locs_ring['angle']/180*np.pi, df_locs_ring['radius'], alpha=0.5, c='#1f77b4')
    
    # set maximum radius for plot such that 99% of locs are shown
    ax_1.set_rmax(radius_limit)

    ax_1.set_title('Localizations within ring', fontsize = 16)
    
    """
    annotate_y = 0.85
    plt.annotate("Fit radius: {:.1f} nm, defined ring width: $\pm${:.0f} nm\n".expandtabs().format(radius_fit,dR),xy=(0.05,annotate_y-0.02), xycoords = 'subfigure fraction',fontsize=14)
    plt.annotate("Percent of locs on ring: {:.1f}% \t\t Min. percent of locs on ring: {:.1f}%\n".expandtabs().format(perc_on_ring*100,perc_on_ring_min*100),xy=(0.05,annotate_y-0.04), xycoords = 'subfigure fraction',fontsize=14)
    plt.annotate("Percent of locs inside of ring: {:.1f}%  \t Max. percent of locs inside of ring: {:.1f}%\n".expandtabs().format(perc_inside_ring*100,perc_inside_ring_max*100),xy=(0.05,annotate_y-0.06), xycoords = 'subfigure fraction',fontsize=14)
    plt.annotate("Percent of locs outside of ring: {:.1f}% \t Max. percent of locs outside of ring: {:.1f}%\n".expandtabs().format(perc_outside_ring*100,perc_outside_ring_max*100),xy=(0.05,annotate_y-0.08), xycoords = 'subfigure fraction',fontsize=14)
    """
    
    
    ax2 = fig.add_subplot(gs[0, 1])
    
    plot_cum_exp(kinetics_all, fit_result_len_all, fit_result_dark_all,'all locs', ax=ax2)
    
    ax3 = fig.add_subplot(gs[0, 2])
    
    plot_cum_exp(kinetics_ring, fit_result_len_ring, fit_result_dark_ring, 'locs on ring', ax=ax3)
    
    
    
    img_fname = "fov_{}_{}_pick_{}_kinetics_on_ring".format(ring['fov_id'], ring['cell_type'], ring['group'])

    plt.close(fig)
    if filtered:
        if ring['filter_passed'] == 'Yes':
            img_name = os.path.join(analysis_folder, img_fname)
            fig.savefig(img_name+img_format, dpi=dpi, format="png")
        elif ring['filter_passed'] == 'No':
            img_name = os.path.join(excluded_folder, img_fname)
            fig.savefig(img_name+img_format, dpi=dpi, format="png")
    else:
        img_name = os.path.join(analysis_folder, img_fname)
        fig.savefig(img_name+img_format, dpi=dpi, format="png")
        
    

df_ring_data = df_ring_data.drop(columns='filename_ring')


columns = ["fov_id",
            "cell_type",
            "filename",
            "group",
            "residual",
            "radius",
            "angle_between_ring_and_coverslip",
            "Sigma",
            "n_events_all",
            "mean_bright_all",
            "mean_dark_all",
            "fit_bright_all",
            "fit_dark_all",
            "n_events_ring",
            "mean_bright_ring",
            "mean_dark_ring",
            "fit_bright_ring",
            "fit_dark_ring",
           ]

if filtered:
    columns.append('filter_passed')
    
# Save ring data
df_ring_data = pd.DataFrame(ring_data, columns=columns)

if filtered:
    df_ring_data.to_csv(os.path.join(path,"ring_data_filter_kinetics_on_ring.csv"))
    # Save dataframe with cell means for easy loading of data for postprocessing
    df_ring_data.to_pickle(os.path.join(path,"ring_data_filter_kinetics_on_ring.pkl"))
else:
    df_ring_data.to_csv(os.path.join(path,"ring_data_kinetics_on_ring.csv"))
    # Save dataframe with cell means for easy loading of data for postprocessing
    df_ring_data.to_pickle(os.path.join(path,"ring_data_kinetics_on_ring.pkl"))



Processing rings:   0%|          | 0/308 [00:00<?, ?it/s]